In [1]:
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np

In [5]:
class BaseModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim, bias=False)
    def forward(self, x):
        return self.linear(x)

In [21]:
class RobustNoiseAddition:
    def __init__(self, input_dim, output_dim, c, model_lr=0.001, model_iterations=10):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.c = torch.abs(c)
        self.model_lr = model_lr
        self.model_iterations = model_iterations
        self.model = BaseModel(input_dim, output_dim)

    def find_largest_delta(self, X):
        """
        Find Delta that maximizes ||Delta||_2 subject to
        sum_{i=1}^n Delta_{i,j}^2 <= c_j for each feature j
        """
        n, p = X.shape
        Delta = torch.zeros((n, p))
        for j in range(p):
            magnitude = torch.sqrt(self.c[j] / n)
            signs = torch.sign(torch.randn(n))
            Delta[:, j] = magnitude * signs
        return Delta

    def fit(self, X, y):
        """Train robust model"""
        X_mean = X.mean(0)
        X_std = X.std(0)
        y_mean = y.mean()
        y_std = y.std()

        X_normalized = (X - X_mean) / X_std
        y_normalized = (y - y_mean) / y_std

        Delta = self.find_largest_delta(X_normalized)
        X_perturbed = X_normalized + Delta

        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.model_lr)
        criterion = nn.MSELoss()

        for i in tqdm(range(self.model_iterations)):
            optimizer.zero_grad()
            y_pred = self.model(X_perturbed)
            loss = criterion(y_pred, y_normalized.view(-1, 1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(
                self.model.parameters(), max_norm=1.0)
            optimizer.step()

        normalized_weights = self.model.linear.weight.data
        denormalized_weights = normalized_weights * (y_std / X_std)
        self.denormalized_weights = denormalized_weights
        return self

    def predict(self, X):
        """Make predictions"""
        self.model.eval()
        with torch.no_grad():
            return self.model(X)

In [23]:
torch.manual_seed(42)
np.random.seed(42)

n_samples, n_features = 100, 10
X = torch.randn((n_samples, n_features))
true_weights = torch.FloatTensor([i + 1 for i in range(n_features)])
y = X @ true_weights + torch.normal(0, 0.1, (n_samples,))

c = torch.abs(torch.randn((n_features,))) * 0.1

robust_model = RobustNoiseAddition(
    input_dim=n_features,
    output_dim=1,
    c=c,
    model_lr=0.001,
    model_iterations=1000
)
robust_model.fit(X, y)
print("Noise Addition Weights:", robust_model.denormalized_weights)
print("True Weights:", true_weights)

100%|██████████| 1000/1000 [00:00<00:00, 9432.63it/s]

Noise Addition Weights: tensor([[0.9465, 2.0440, 2.9547, 4.1448, 4.8771, 6.0621, 6.4449, 8.0948, 8.8365,
         9.9975]])
True Weights: tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])
